In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.chdir("..")

# import sys
# sys.path.append("./vrevals")

import re
import ast
import json
import numpy as np
import pandas as pd
import swifter
import yaml
import time
import glob

import matplotlib.pyplot as plt

from collections import Counter
from src.utils.math_equivalence import is_math_equiv, extract_boxed_answer
from src.utils.pass_k_utils import estimate_pass_at_k

In [84]:
class Args:
    dataset_name = "legostar"
    # split = "test"
    split = "train"
    k_list = [1,4,8,32]
    subset_num = None
    step_by_step_prompt = True
    n_threads = 1
args = Args()

# job_dir = f"runs/{args.dataset_name}_prompt2.qwen2.5-0.5b"
# job_dir = f"runs/{args.dataset_name}.qwen2.5-0.5b"


# job_dir = f"runs/aime24.qwen2.5-0.5b"
# job_dir = f"runs/gsm8k_prompt2.qwen2.5-0.5b"
# job_dir = f"runs/gsm8k.qwen2.5-0.5b"
# job_dir = f"runs/gsm8k.gemma-2b"

job_dir = f"runs/{args.dataset_name}.qwen2.5-0.5b"

# sampler_config_dir = f'{job_dir}/finished/sft_ep2.direct/sample_1'
# sampler_config_dir = f'{job_dir}/sft_ep1.prefix/sample_1'
sampler_config_dir = f'{job_dir}/sft_ep9.direct/sample_2'

# sampler_config_dir = f'{job_dir}/finished/sft_ep1.direct/sample_1'
# sampler_config_dir = f'{job_dir}/instruct.prefix/Anya'
# sampler_config_dir = f'{job_dir}/instruct.prefix/Anya'
# sampler_config_dir = f'{job_dir}/instruct.prefix/Becky'

prompt_csv_path = f'{job_dir}/{args.split}.prompts.csv'

# with open(f"{sampler_config_dir}/sampler_config.yaml", "r") as f:
#     sampler_config = yaml.safe_load(f)
# sampler_config

print(sampler_config_dir)

runs/legostar.qwen2.5-0.5b/sft_ep9.direct/sample_2


In [85]:
with open("/data/hnn5071/lrm/datasets/legostar/train.json", "r") as f:
    testset = json.load(f)

for e in testset:
    equations = e['answer'].split(",")
    equations = [f'{eq[0]}={int(eq[-2:])}' for eq in equations]
    e['answer'] = ','.join(equations)

testset = {e['id']: e for e in testset}

def extract_lego_equations(text):
    """
    Extract equations like a = -1, f = 1, b = 1 from a text.
    Returns a list of strings in the form 'a=-1'.
    """
    pattern = r'\b([a-zA-Z])\s*=\s*(-?\d+)\b'
    matches = re.findall(pattern, text)
    return ','.join([f"{var}={val}" for var, val in matches])

In [86]:
all_generation_csv = glob.glob(f"{sampler_config_dir}/{args.split}.generations*.csv")
# all_generation_csv = glob.glob(f"{sampler_config_dir}/*generations*.csv")
all_generation_df = [pd.read_csv(p) for p in all_generation_csv]
# Concatenate all dataframes in all_generation_df into a single dataframe
generation_df = pd.concat(all_generation_df, ignore_index=True)

generation_df['pred_answer'] = generation_df.response.apply(extract_boxed_answer)
generation_df['equations'] = generation_df.response.apply(extract_lego_equations)

generation_df['is_valid'] = generation_df.pred_answer.apply(lambda x: len(x) > 0)
# generation_df['is_correct'] = generation_df.swifter.progress_bar(True).apply(lambda row: is_math_equiv(row['gt_answer'], row['pred_answer'], 1), axis=1)


def is_number(s):
    try:
        return float(s)
    except ValueError:
        return float('nan')

# generation_df['is_correct'] = generation_df.swifter.progress_bar(True).apply(lambda row: is_number(row['gt_answer']) == is_number(row['pred_answer']), axis=1)
generation_df['is_correct'] = generation_df.swifter.progress_bar(True).apply(lambda row: testset[row['question_id']]['answer'] == row['equations'], axis=1)

Pandas Apply:   0%|          | 0/64000 [00:00<?, ?it/s]

In [87]:
generation_df

,question_id,prompt_id,response,pred_answer,gt_answer,sampler_config,equations,is_valid,is_correct
0,20952,20952,Evaluating the variables according to their de...,-1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","b=1,n=1,t=1,e=-1",True,False
1,20952,20952,"Using the dependency rules, we evaluate each v...",1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","z=1,p=-1,c=1,e=1",True,True
2,20952,20952,"Starting from the given value, we propagate th...",-c,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","b=1,n=1,t=1",True,False
3,20952,20952,"Starting from the given value, we propagate th...",1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","z=1,p=-1,c=1,e=1",True,True
4,3648,3648,By following the signed relationships between ...,-1,-1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","q=-1,c=1,l=-1,w=-1",True,True
...,...,...,...,...,...,...,...,...,...
63995,4507,4507,"Using the dependency rules, we evaluate each v...",1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","o=-1,m=1,j=1,q=1",True,True
63996,5077,5077,By following the signed relationships between ...,1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","c=1,y=1,s=-1,e=1",True,False
63997,5077,5077,"Starting from the given value, we propagate th...",1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","c=1,y=1,s=-1,e=1",True,False
63998,5077,5077,By following the signed relationships between ...,1,1,"{'sampler': {'class': 'VLLMSampler', 'model_na...","r=1,u=-1,h=-1,e=1",True,True


In [88]:
# print(generation_df[generation_df['question_id'] == 179].response.values[5])

In [89]:
# corrects_grouped.iloc[10].corrects

In [90]:
# Filter the dataframe to keep only rows where the response is valid
# valid_generation_df = generation_df[generation_df['is_valid']]
valid_generation_df = generation_df

cut_off_size = 32

grouped_df = valid_generation_df.groupby(['question_id', 'prompt_id'])
corrects_grouped = grouped_df.is_correct.apply(list).reset_index(name='corrects')
corrects_grouped.corrects = corrects_grouped.corrects.apply(lambda x: x[:cut_off_size])
corrects_grouped['num_samples'] = corrects_grouped.corrects.apply(len)
corrects_grouped['num_math_equal'] = corrects_grouped.corrects.apply(sum)

corrects_grouped = corrects_grouped.sort_values(by=['question_id'])

min_num_samples = corrects_grouped['num_samples'].min()

k_list = range(1,2000)
detail_pass_at_k = {
    f"pass@{k}": estimate_pass_at_k(corrects_grouped['num_samples'].values, 
                                    corrects_grouped['num_math_equal'].values, k)
    for k in k_list
    if (min_num_samples >= k).all()
}
pass_at_k = {k: detail_pass_at_k[k].mean() for k in detail_pass_at_k}
pass_at_k

{'pass@1': np.float64(0.816578125),
 'pass@2': np.float64(0.9283084677419355),
 'pass@3': np.float64(0.9646342741935483),
 'pass@4': np.float64(0.9799817436040044),
 'pass@5': np.float64(0.9875807767559193),
 'pass@6': np.float64(0.9917698876176351),
 'pass@7': np.float64(0.9942660088548054),
 'pass@8': np.float64(0.9958430958900203),
 'pass@9': np.float64(0.9968855421800576),
 'pass@10': np.float64(0.9975993119832763),
 'pass@11': np.float64(0.998101751159935),
 'pass@12': np.float64(0.9984632007883865),
 'pass@13': np.float64(0.9987276695739112),
 'pass@14': np.float64(0.9989237140385665),
 'pass@15': np.float64(0.9990704511107492),
 'pass@16': np.float64(0.9991810244666274),
 'pass@17': np.float64(0.9992646799504534),
 'pass@18': np.float64(0.9993280515111714),
 'pass@19': np.float64(0.9993759829215576),
 'pass@20': np.float64(0.9994120673246326),
 'pass@21': np.float64(0.9994390098646397),
 'pass@22': np.float64(0.9994588762070578),
 'pass@23': np.float64(0.9994732655585978),
 'pas

In [91]:
overall_results = {
    'detail_pass_at_k': {k:v.tolist() for k,v in detail_pass_at_k.items()},
    'pass_at_k': pass_at_k,
}
final_metrics = {'overall': overall_results}
t = time.localtime()
metrics_json_name = f'metrics.{t.tm_mon}.{t.tm_mday},{t.tm_hour}:{t.tm_min}.json'
with open(os.path.join(sampler_config_dir, metrics_json_name), mode='w', encoding='utf-8') as json_file:
    json.dump(final_metrics, json_file, indent=4, ensure_ascii=False)


In [19]:
final_metrics

{'overall': {'detail_pass_at_k': {'pass@1': [0.4999999999999999,
    0.4999999999999999,
    0.875,
    0.75,
    0.4999999999999999,
    0.75,
    0.75,
    0.0,
    0.4999999999999999,
    0.25,
    0.3749999999999999,
    0.75,
    0.4999999999999999,
    0.3749999999999999,
    0.0,
    0.3749999999999999,
    0.75,
    0.6249999999999999,
    0.25,
    0.3749999999999999,
    1.0,
    1.0,
    0.0,
    0.75,
    0.25,
    0.75,
    1.0,
    0.25,
    0.3749999999999999,
    0.125,
    0.25,
    0.3749999999999999,
    0.4999999999999999,
    0.3749999999999999,
    0.75,
    0.125,
    0.6249999999999999,
    0.25,
    0.3749999999999999,
    0.25,
    0.3749999999999999,
    0.125,
    0.3749999999999999,
    0.4999999999999999,
    0.6249999999999999,
    0.875,
    0.3749999999999999,
    0.0,
    0.25,
    0.25,
    0.875,
    0.3749999999999999,
    0.875,
    0.0,
    0.6249999999999999,
    0.6249999999999999,
    0.25,
    0.125,
    0.875,
    0.4999999999999999,
    0.62

In [43]:
responses = grouped_df.response.apply(list).reset_index(name='text')
pred_answers = grouped_df.pred_answer.apply(list).reset_index(name='text')

In [12]:
responses.text[0]

["Okay, so Janet has these ducks that lay 16 eggs each day. Hmm, let me figure out how many eggs she has in total. If she has 16 eggs per day, and she's selling them at the farmers' market for $2 each, then I guess I need to calculate how much she makes from selling the eggs.\n\nFirst, let me think about how many eggs she eats for breakfast. She eats three eggs every morning. So, that's 3 eggs per day. Then, she also bakes muffins for her friends every day, using four eggs. So, that's another 4 eggs per day. \n\nSo, to find the total number of eggs she has left after eating and baking, I should subtract the eggs she eats and the eggs she uses for muffins from the total number of eggs laid each day. \n\nLet me write that down:\n\nTotal eggs laid per day = 16\n\nEggs eaten for breakfast = 3\n\nEggs used for muffins = 4\n\nSo, the number of eggs left would be:\n\nTotal eggs left = Total eggs laid - Eggs eaten - Eggs used\n\nPlugging in the numbers:\n\nTotal eggs left = 16 - 3 - 4\n\nLet m

In [23]:
print(generation_df[~generation_df['is_valid']].response.values[0])

Okay, so Kylar went to the store to buy glasses, and he wants to buy 16 glasses in total. The problem says that one glass costs $5, but every second glass costs only 60% of the price. Hmm, okay, so I need to figure out how much he needs to pay for all 16 glasses.

First, let me break this down. There are 16 glasses in total. The first glass is $5, and each subsequent glass is 60% of the original price. So, the first glass is $5, the second glass is 60% of $5, the third is 60% of $5, and so on, until the sixteenth glass.

Wait, hold on, is that right? The problem says every second glass costs only 60% of the price. So, does that mean the first glass is $5, and then every second glass is 60% of $5? Or is it that every second glass is 60% of the total number of glasses? Hmm, that might be a bit confusing.

Let me read the problem again: "One glass costs $5, but every second glass costs only 60% of the price." So, it seems like the first glass is $5, and each subsequent glass is 60% of the